# 🗺️ NYC Walksheds / Isochrones in Python
This notebook demonstrates how to calculate walksheds (or isochrones) for a set of origin points in New York City using `osmnx`, `networkx`, and `geopandas`. We'll generate polygons representing areas reachable within a given walking distance.

In [ ]:
# 📦 Install required packages
!pip install osmnx networkx geopandas shapely

## 1. Load or define origin points
We'll define a few example origin points in Manhattan.

In [ ]:
import geopandas as gpd
from shapely.geometry import Point

# Define example origin points
origins = gpd.GeoDataFrame({
    'id': [1, 2],
    'name': ['Times Square', 'Midtown'],
    'geometry': [Point(-73.9851, 40.7580), Point(-73.9750, 40.7450)]
}, crs="EPSG:4326")

## 2. Download the NYC walking network with `osmnx`

In [ ]:
import osmnx as ox

# Download the street network for Manhattan
graph = ox.graph_from_place("Manhattan, New York City, USA", network_type='walk')

## 3. Define a function to generate walksheds (isochrones) based on walking distance

In [ ]:
import networkx as nx
from shapely.geometry import Polygon

def make_isochrone(graph, origin_point, distance=500, edge_buff=25, node_buff=0, infill=True):
    orig_node = ox.distance.nearest_nodes(graph, origin_point.x, origin_point.y)
    lengths = nx.single_source_dijkstra_path_length(graph, orig_node, cutoff=distance, weight='length')
    subgraph = graph.subgraph(lengths.keys())

    node_points = [Point((data['x'], data['y'])) for node, data in subgraph.nodes(data=True)]
    node_gdf = gpd.GeoDataFrame(geometry=node_points, crs="EPSG:4326")

    nodes_proj = node_gdf.to_crs(epsg=32618)
    buffered = nodes_proj.buffer(edge_buff)
    unioned = buffered.unary_union
    if infill:
        unioned = unioned.buffer(0)
    
    polygon = gpd.GeoSeries(unioned).set_crs(epsg=32618).to_crs(epsg=4326)
    return polygon.iloc[0]

## 4. Calculate walksheds for each origin

In [ ]:
walksheds = []
for idx, row in origins.iterrows():
    poly = make_isochrone(graph, row.geometry, distance=800)
    walksheds.append({'id': row['id'], 'geometry': poly})

walksheds_gdf = gpd.GeoDataFrame(walksheds, crs="EPSG:4326")

## 5. Export the results to GeoJSON and Shapefile

In [ ]:
walksheds_gdf.to_file("nyc_walksheds.geojson", driver="GeoJSON")
walksheds_gdf.to_file("nyc_walksheds.shp")

## 6. Visualize the result (optional)

In [ ]:
import matplotlib.pyplot as plt

ax = walksheds_gdf.plot(color='lightblue', edgecolor='black')
origins.plot(ax=ax, color='red', markersize=10)
plt.title("NYC Walksheds (800m)")
plt.show()